In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-17 02:27:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-12-17 02:27:23 (8.77 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz" 
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [6]:
dropna_df = df.dropna()

In [7]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
from pyspark.sql.functions import count
customers_df = dropna_df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count").drop_duplicates()
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15634680|             1|
|   44696507|             1|
|   19901367|             1|
|   11337682|             1|
|   52021773|             1|
|   29916198|             1|
|   42030944|             1|
|   52765209|             1|
|   13519509|             2|
|   39320350|             5|
|   23249582|             1|
|   46351928|             3|
|   44179620|             1|
|     637252|             1|
|   34038793|             1|
|   51136490|             1|
|   49354815|             1|
|     149761|             2|
|   43333798|             1|
|    4657027|             1|
+-----------+--------------+
only showing top 20 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = dropna_df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])

In [9]:
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|031024837X|The Expositor's B...|
|0838447384|Triple Play Plus ...|
|097680428X|College Prep Geni...|
|1618945157|   Trainz: A New Era|
|1881901912|Learn Sicilian / ...|
|9650706666|Encyclopedia Juda...|
|B00003GPSY|       VideoWave 3.0|
|B00004RRSZ|L&H Voice Xpress ...|
|B00004S85T|         Monarch 5.0|
|B00004TR44|Seagate Crystal R...|
|B00004U9SS|PlanWrite Busines...|
|B00004UB51|NetObjects Fusion...|
|B00004X02L|Quicken Expensabl...|
|B000059OSD|Instant Immersion...|
|B000059P9E|Essential Christi...|
|B00005NCJ3|Jasc Paint Shop P...|
|B00005RFCX|Body Works 4.0 Du...|
|B000062YLB|30,000 Business C...|
|B000068CO9|      Keyboard Coach|
|B00006CJSP|SuSE StarOffice 6...|
+----------+--------------------+
only showing top 20 rows



In [10]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df = dropna_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])


In [11]:
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
|R2C1DJSCC8UFS6|   37107850|B00EP7AP7C|     279360628| 2015-08-31|
|R1AXGS1W4YFXMX|     302120|B00OW2PET4|     729971168| 2015-08-31|
|R1XU1B93402SYJ|   20193077|B00N4OLCRO|     776572654| 2015-08-31|
|R2U432NB3OPVR0|   13106017|B005CELN8W|     222071424| 2015-08-31|
|R3R6FIMIOQ5SP9|   32587108|B005CELL1G|     168801430| 2015-08-31|
|R12TX6V09C9QNQ|   32020017|B00H09BOXQ|     455502995| 2015-08-31|
|R33UCII6YKUMKV|   13376158|B00MCLGAAO|     652069521| 2015-08

In [12]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = dropna_df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine','verified_purchase']).drop_duplicates()

In [13]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R3F8SH5XNMIYIH|          4|            0|          0|   N|                Y|
| R5L0WP3MZVTM8|          4|            0|          0|   N|                Y|
|R3UWNSUI0S5571|          1|            1|          2|   N|                Y|
| RD4048QLYIB8Y|          5|            1|          2|   N|                N|
|R138WLFRO0RAZS|          5|            0|          0|   N|                Y|
|R3LTBVRHLYAJWT|          5|            0|          0|   N|                Y|
|R1YAQ9K1ARUYY3|          5|            1|          1|   N|                Y|
|R123ZJH7X500FY|          3|            1|          2|   N|                Y|
|R3TUJRPUVB38KJ|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<database>.ca5cq2n6suy3.us-east-2.rds.amazonaws.com:5432/<databasename>"
config = {"user":"username", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [15]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [16]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [17]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [18]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)